This notebook's progressed is stopped once we reached the limitations of Jupyter Notebook's Plotly. We had to switch to Plotly Dash to continue the project. Refer to the orb15 `main.py` for the complete project.

In [ ]:
from resources.utilities import *

df = load_file("./Data/SPY_5min_merged.csv", date_format="%Y-%m-%d %H:%M:%S")
print(df.dtypes)
print(df.head())
print(df.tail())

In [2]:
df['date'] = df['timestamp'].dt.date

df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['time_val'] = df['hour'] * 100 + df['minute']  # Creates time in HHMM format

df_regular_hours = df[(df['time_val'] >= 930) & (df['time_val'] <= 1555)]

print(df.shape)
print(df_regular_hours.shape)
print(f"Trading Days Available: {df_regular_hours['date'].nunique()}")

(96541, 10)
(39156, 10)
Trading Days Available: 502


In [81]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

TRADING_DAY_TO_DISPLAY = 150
df_working = df_regular_hours[df_regular_hours['date'] == df_regular_hours['date'].unique()[TRADING_DAY_TO_DISPLAY]].reset_index(drop=True)

# Set Custom Views (xrange and yrange)
xrange = [df_working['timestamp'].iloc[0] - pd.Timedelta(minutes=5), df_working['timestamp'].iloc[-1] + pd.Timedelta(minutes=5)]
padding = 0.05
y_min, y_max = df_working['low'].min(), df_working['high'].max()
y_range = y_max - y_min
yrange = [y_min - (y_range * padding), y_max + (y_range * padding)]

# Set colors for candlestick
colors = {
    'bearish_wick': '#FF6347',
    'bearish_body': '#FF6347',
    'bearish_border': '#CC503A',
    
    'bullish_body': '#32CD32',
    'bullish_wick': '#32CD32',
    'bullish_border': '#28A028',
    
    'background': '#FAFAD2',
    'foreground': '#228B22',
    'text': '#000000',
    'grid': '#D3D3D3'
}

fig = make_subplots()

date_str = df_working['date'].iloc[0].strftime("%Y-%m-%d")
dayofweek = df_working['date'].iloc[0].strftime('%a')
opening_range = df_working.head(3)
orh, orl = opening_range['high'].max(), opening_range['low'].min()
ormid = (orh + orl) / 2

fig.add_shape(
    type="line",
    x0=date_str + " 09:30:00",
    x1=date_str + " 16:00:00",
    y0=orh,
    y1=orh,
    line=dict(
        color="#000000",
        width=1,
        dash="solid",
    ),
    name="ORH"
)

fig.add_shape(
    type="line",
    x0=date_str + " 09:30:00",
    x1=date_str + " 16:00:00",
    y0=orl,
    y1=orl,
    line=dict(
        color="#000000",
        width=1,
        dash="solid",
    ),
    name="ORL"
)

fig.add_shape(
    type="line",
    x0=date_str + " 09:30:00",
    x1=date_str + " 16:00:00",
    y0=ormid,
    y1=ormid,
    line=dict(
        color="#000000",
        width=1,
        dash="dash",
    ),
    name="OR50"
)

fig.add_shape(
    type="rect",
    x0=date_str + " 09:30:00",
    x1=date_str + " 16:00:00",
    y0=orl,
    y1=orh,
    fillcolor="rgba(214, 180, 252, 0.2)",
    line=dict(width=0),
    layer="below"
)

fig.update_layout(
    paper_bgcolor=colors['background'],
    plot_bgcolor=colors['background'],
    font={'color': colors['text'], 'family': 'Inconsolata', 'size': 15},
    title = f"SPY: {date_str} {dayofweek}", title_y = 0.95,

    xaxis=dict(
        type='date',
        rangeslider=dict(visible=False), # cleaner
        fixedrange=True,  # Allow zooming on x-axis
        # autorange=True, # this must be disabled IF we set an initial range window
        range=xrange,
        # rangebreaks no longer needed since we are plotting day by day
        # rangebreaks=[
        #     dict(bounds=["sat", "mon"]),  # Hide weekends
        #     dict(bounds=[17, 9], pattern="hour")  # Hide non-trading hours
        # ],
        showgrid=False
    ),
    yaxis=dict(
        # autorange=True,  # Enable autorange for y-axis
        range=yrange,
        fixedrange=True,  # VERY IMPORTANT TO OVERRIDE THE DEFAULT BEHAVIOR - allow y-axis zooming

        showgrid=True,
        gridcolor=colors['grid'],
        griddash='dot',
        gridwidth=1,

        tickformat=",.2f",
    ),
    margin=dict(l=20, r=20, t=40, b=20),
)

# add the initial trace (idk why, but this is needed to make the animation work)
fig.add_trace(go.Candlestick(
    x=df_working['timestamp'][:1],
    open=df_working['open'][:1],
    close=df_working['close'][:1],
    high=df_working['high'][:1],
    low=df_working['low'][:1],
    name="Price",
    increasing={'line': {'color': colors['bullish_border'], 'width': 0.6}, 'fillcolor': colors['bullish_body']},
    decreasing={'line': {'color': colors['bearish_border'], 'width': 0.6}, 'fillcolor': colors['bearish_body']}
))

# add the rest of the frames
frames = []
for i in range(0, 78):
    frame = go.Frame(
        data = [go.Candlestick(
            x=df_working['timestamp'][:i],
            open=df_working['open'][:i],
            close=df_working['close'][:i],
            high=df_working['high'][:i],
            low=df_working['low'][:i],
            name="Price",
            increasing={
                'line': {'color': colors['bullish_border'], 'width': 0.6},
                'fillcolor': colors['bullish_body']
            },
            
            decreasing={
                'line': {'color': colors['bearish_border'], 'width': 0.6},
                'fillcolor': colors['bearish_body']
            }
        )],
        name = f"Frame {i}"
    )
    frames.append(frame)
fig.frames = frames

fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        x=0.3, y=1.08,
        buttons=[dict(
            label="▶",
            method="animate",
            args=[None, dict(frame=dict(duration=800, redraw=True), fromcurrent=True, mode="immediate", transition=dict(duration=0))]
        ),
        dict(
            label="▶▶",
            method="animate",
            args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True, mode="immediate", transition=dict(duration=0))]
        ),
         dict(
            label="⏸",
            method="animate",
            # the [None] in brackets is EXTREMELY IMPORTANT for some reason 
            # this is some deep deep undocumented shit
            # args=[None, ...]	Restarts from frame 0
            # args=[[None], ...]	Pauses but keeps current frame
            args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))]
        )]
    )]
)

fig.show(config={'displayModeBar': True, 'scrollZoom': False, 'displaylogo': False})


In [4]:
df = df_regular_hours

for day, group in df.groupby(df["timestamp"].dt.date):
    opening_range = group[group['timestamp'].dt.strftime('%H:%M').isin(['09:30', '09:35', '09:40'])]
    ORH = opening_range['high'].max()
    ORL = opening_range['low'].min()

    # for every following candle, wait for a break and close outside the OR
    # how do we quantify how significant the break is?

    break

##### ARCHIVE #####

    ### IN THE PAST, WHEN WE DISPLAYED MULTIPLE DAYS AT ONCE AND WANT TO SEE OPENING RANGE ON ALL DAYS

    # def calculate_opening_ranges(df):
    #     opening_range = df[df['time_val'] < 945].groupby('date').head(3)
    #     opening_range_hl = opening_range.groupby('date').agg({
    #         'high': 'max',
    #         'low': 'min'
    #     }).reset_index()
        
    #     # Create dictionary with date as key and high/low values
    #     ranges = {}
    #     for _, row in opening_range_hl.iterrows():
    #         ranges[row['date']] = {
    #             'high': row['high'],
    #             'mid': (row['high'] + row['low']) / 2,
    #             'low': row['low'],
    #             'interval': (row['high'] - row['low']) / 2
    #         }
    #     return ranges

    # opening_ranges = calculate_opening_ranges(df_working)

    # for date, range_values in opening_ranges.items(): blah blah blah

Todo:
1. do the ORB 15 lines - ORH, ORL, OR50, ORH + 50, ORL - 50, etc
2. implement interactive entry button with interactive TP and SL setting
3. implement trade logging
4. implement dynamic chart plotting.
5. once we done here we can move on the ORB report